In [1]:
import skmob
import pandas as pd
import pymysql, csv, os
from datetime import datetime, timedelta
from skmob.preprocessing import filtering, detection, compression

conn = pymysql.connect(
    host='127.0.0.1', user='root', password='00000000', db='USERS', charset='utf8')

with conn:
    with conn.cursor() as cur:
        sql = "SELECT distinct deviceid from tc_positions"
        cur.execute(sql)
        device_id_list = cur.fetchall()

        sum_of_filtering = 0
        file = open('filtered_stay_point.txt', 'w')
        for device_id in device_id_list:
            device_id = str(device_id[0])
            sql = "SELECT * FROM tc_positions WHERE deviceid=" + device_id
            cur.execute(sql)
            trajectory_data = cur.fetchall()

            data_list = list()
            for data in trajectory_data:
                data_list.append(list([data[1], data[3], data[4], data[2]]))

            tdf = skmob.TrajDataFrame(
                data_list, user_id=0, latitude=1, longitude=2, datetime=3)
            tdf['datetime']=tdf['datetime']+timedelta(hours=9)

            stdf = detection.stay_locations(
                tdf, stop_radius_factor=0.5, minutes_for_a_stop=30.0, spatial_radius_km=0.2, leaving_time=True)
            
            ftdf = filtering.filter(tdf, max_speed_kmh=200, speed_kmh=3)
            fstdf = detection.stay_locations(
                ftdf, stop_radius_factor=0.5, minutes_for_a_stop=30.0, spatial_radius_km=0.2, leaving_time=True)

            file.write("device id: " + device_id + ' filtered: %d\n' % (len(stdf) - len(fstdf)))
        
            print('Device id:\t\t\t\t%s' % device_id)
            print('Points of the original trajectory:\t%s' % len(ftdf))
            print('Points of stops:\t\t\t%s' % len(fstdf))

            if(len(fstdf)==0): continue

            stay_point_list = fstdf.values.tolist()
            for stay_point in stay_point_list:
                stay_point[3] = stay_point[3].strftime('%Y-%m-%d %H:%M:%S')
                stay_point[4] = stay_point[4].strftime('%Y-%m-%d %H:%M:%S')
            fields = ['deviceid', 'latitude', 'longitude', 'datetime', 'leaving_datetime']
            path = "./staypoint/"+device_id
            if not os.path.isdir(path):
                os.mkdir(path)
            with open("./staypoint/"+device_id+'/stay_points.csv', 'w',newline='') as f:
                write = csv.writer(f)
                write.writerow(fields)
                write.writerows(stay_point_list)
            
            stay_point_dict = dict()
            for idx in range(len(stay_point_list)):
                date_time = stay_point_list[idx][3][0:10]
                if date_time not in stay_point_dict.keys():
                    stay_point_dict[date_time] = list()
                stay_point_dict[date_time].append(stay_point_list[idx])
            
            for key in stay_point_dict.keys():
                with open("./staypoint/"+device_id+'/'+key+'.csv', 'w', newline='') as f:
                    write = csv.writer(f)
                    write.writerow(fields)
                    write.writerows(stay_point_dict[key])    

Device id:				51
Points of the original trajectory:	9216
Points of stops:			56
Device id:				52
Points of the original trajectory:	8401
Points of stops:			47
Device id:				54
Points of the original trajectory:	10911
Points of stops:			43
Device id:				53
Points of the original trajectory:	30799
Points of stops:			114
Device id:				58
Points of the original trajectory:	9088
Points of stops:			46
Device id:				59
Points of the original trajectory:	6684
Points of stops:			72
Device id:				60
Points of the original trajectory:	8550
Points of stops:			59
Device id:				61
Points of the original trajectory:	8728
Points of stops:			37
Device id:				62
Points of the original trajectory:	9364
Points of stops:			1
Device id:				63
Points of the original trajectory:	4391
Points of stops:			19
Device id:				64
Points of the original trajectory:	14990
Points of stops:			88
Device id:				65
Points of the original trajectory:	7279
Points of stops:			57
Device id:				79
Points of the original trajector